<a href="https://colab.research.google.com/github/Alienor1268/flight/blob/main/airline_million.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle.json


In [ ]:
! kaggle datasets download -d bulter22/airline-data
! unzip airline-data.zip

100% 4.20G/4.20G [01:19<00:00, 42.5MB/s]
100% 4.20G/4.20G [01:19<00:00, 56.6MB/s]
Archive:  airline-data.zip
  inflating: airline.csv.shuffle     
  inflating: carriers.csv            


In [ ]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
!pip install -q gwpy

In [ ]:
%%capture
%%R
install.packages('sparklyr')
library(sparklyr)
install.packages('dplyr')
library(dplyr)
library(data.table)
spark_install(version = "3.0.0")


In [ ]:
%%R
# only import the first couple of rows for exploratory analysis 
df_10000 <- fread('/content/airline.csv.shuffle', nrows=10000)


In [ ]:
%%R
df_10000_naomit <- na.omit(df_10000)
sum(is.na(df_10000_naomit))
str(df_10000_naomit)

Classes ‘data.table’ and 'data.frame':	2672 obs. of  29 variables:
 $ ActualElapsedTime: int  45 100 33 57 95 143 81 119 64 64 ...
 $ AirTime          : int  29 81 22 31 74 122 60 100 50 56 ...
 $ ArrDelay         : int  2 -2 -3 -9 136 4 -6 -18 -24 -1 ...
 $ ArrTime          : int  1120 1220 1916 1659 2343 824 1049 1547 1956 2144 ...
 $ CRSArrTime       : int  1118 1222 1919 1708 2127 820 1055 1605 2020 2145 ...
 $ CRSDepTime       : int  1030 1045 1845 1612 2035 600 930 1450 1900 2040 ...
 $ CRSElapsedTime   : int  48 97 34 56 112 140 85 135 80 65 ...
 $ CancellationCode : chr  "" "" "" "" ...
 $ Cancelled        : int  0 0 0 0 0 0 0 0 0 0 ...
 $ CarrierDelay     : int  0 0 0 0 136 0 0 0 0 0 ...
 $ DayOfWeek        : int  1 1 2 5 7 4 2 3 2 7 ...
 $ DayofMonth       : int  19 6 6 4 30 21 21 3 10 11 ...
 $ DepDelay         : int  5 -5 -2 -10 153 1 -2 -2 -8 0 ...
 $ DepTime          : int  1035 1040 1843 1602 2308 601 928 1448 1852 2040 ...
 $ Dest             : chr  "CMH" "ATL" "HNL" "A

In [ ]:
%%R
df_10000_int = select(df_10000_naomit, -8, -15, -22, -24, -27)
str(df_10000_int)

Classes ‘data.table’ and 'data.frame':	2672 obs. of  24 variables:
 $ ActualElapsedTime: int  45 100 33 57 95 143 81 119 64 64 ...
 $ AirTime          : int  29 81 22 31 74 122 60 100 50 56 ...
 $ ArrDelay         : int  2 -2 -3 -9 136 4 -6 -18 -24 -1 ...
 $ ArrTime          : int  1120 1220 1916 1659 2343 824 1049 1547 1956 2144 ...
 $ CRSArrTime       : int  1118 1222 1919 1708 2127 820 1055 1605 2020 2145 ...
 $ CRSDepTime       : int  1030 1045 1845 1612 2035 600 930 1450 1900 2040 ...
 $ CRSElapsedTime   : int  48 97 34 56 112 140 85 135 80 65 ...
 $ Cancelled        : int  0 0 0 0 0 0 0 0 0 0 ...
 $ CarrierDelay     : int  0 0 0 0 136 0 0 0 0 0 ...
 $ DayOfWeek        : int  1 1 2 5 7 4 2 3 2 7 ...
 $ DayofMonth       : int  19 6 6 4 30 21 21 3 10 11 ...
 $ DepDelay         : int  5 -5 -2 -10 153 1 -2 -2 -8 0 ...
 $ DepTime          : int  1035 1040 1843 1602 2308 601 928 1448 1852 2040 ...
 $ Distance         : int  116 300 100 152 563 780 429 599 214 358 ...
 $ Diverted        

**read 100 000 rows**

In [ ]:
%%R
# only import the first couple of rows for exploratory analysis 
df_100000 <- fread('/content/airline.csv.shuffle', nrows=100000)
df_100000_naomit <- na.omit(df_100000)
df_100000_int = select(df_100000_naomit, -8, -15, -22, -24, -27)


**read million rows**

In [ ]:
%%R
# only import the first couple of rows for exploratory analysis 
df_million <- fread('/content/airline.csv.shuffle', nrows=1000000)
df_million_naomit <- na.omit(df_million)
df_million_int = select(df_million_naomit, -8, -15, -22, -24, -27)

**read five million rows**

In [ ]:
%%R
# only import the first couple of rows for exploratory analysis 
df_five_million <- fread('/content/airline.csv.shuffle', nrows=5000000)
df_five_million_naomit <- na.omit(df_five_million)
df_five_million_int = select(df_five_million_naomit, -8, -15, -22, -24, -27)

In [ ]:
%%R
# connect to spark 
spark_conn <- spark_connect("local")
df_tbl0000 <- copy_to(spark_conn, df_10000_int,  overwrite = TRUE)
df_tbl00000 <- copy_to(spark_conn, df_100000_int,  overwrite = TRUE)
df_million <- copy_to(spark_conn, df_million_int,  overwrite = TRUE)


R[write to console]: * Using Spark: 3.0.0



In [ ]:
%%R
spark_conn <- spark_connect("local")
df_tblfive_million <- copy_to(spark_conn, df_five_million_int,  overwrite = TRUE)

R[write to console]: Re-using existing Spark connection to local



**random forest for 10 000 rows**

In [ ]:
%%R
# training and test set
partitions <- df_tbl0000 %>%
  sdf_random_split(training = 0.7, test = 0.3, seed = 1111)
df_tbl0000_training <- partitions$training
df_tbl0000_test <- partitions$test
rf_model <- df_tbl0000_training %>%
  ml_random_forest(ArrDelay ~ ., type = "regression")
pred <- ml_predict(rf_model, df_tbl0000_test)
ml_regression_evaluator(pred, label_col = 'ArrDelay', metric_name = "rmse",)


[1] 13.32442


**random forest for 100 000 rows**

In [ ]:
%%R
# training and test set - for 100 000 rows
partitions100000 <- df_tbl00000 %>%
  sdf_random_split(training = 0.7, test = 0.3, seed = 1111)
df_tbl00000_training <- partitions100000$training
df_tbl00000_test <- partitions100000$test
rf_model <- df_tbl00000_training %>%
  ml_random_forest(ArrDelay ~ ., type = "regression")
pred_100000 <- ml_predict(rf_model, df_tbl00000_test)
ml_regression_evaluator(pred_100000, label_col = 'ArrDelay', metric_name = "rmse")

[1] 22.55833


**random forest for million rows**

In [ ]:
%%R
# training and test set - for 1 000 000 rows
partition_million <- df_million %>%
  sdf_random_split(training = 0.7, test = 0.3, seed = 1111)
df_million_training <- partition_million$training
df_million_test <- partition_million$test
rf_model <- df_million_training %>%
  ml_random_forest(ArrDelay ~ ., type = "regression")
pred_million <- ml_predict(rf_model, df_million_test)
ml_regression_evaluator(pred_million, label_col = 'ArrDelay', metric_name = "rmse",)


[1] 15.96427


In [ ]:
%%R
# training and test set - for 100 000 rows
partition_five_million <- df_tblfive_million %>%
  sdf_random_split(training = 0.7, test = 0.3, seed = 1111)
df_five_million_training <- partition_five_million$training
df_five_million_test <- partition_five_million$test
rf_model_five_million <- df_five_million_training %>%
  ml_random_forest(ArrDelay ~ ., type = "regression")
pred_five_million <- ml_predict(rf_model_five_million, df_five_million_test)
ml_regression_evaluator(pred_five_million, label_col = 'ArrDelay', metric_name = "rmse",)

In [ ]:
#%%R
#model1 <- ml_random_forest_regressor(
  #df_tbl0000,
  #formula = ArrDelay ~ DayOfWeek ,
  #num_trees = 20,
  #subsampling_rate = 1,
  #max_depth = 5,
)